In [1]:
%%bash
#!/usr/bin/env bash

echo "Update environment..."
apt update -q  &> /dev/null

echo "Install Java..."
apt-get install -q openjdk-11-jdk-headless &> /dev/null

echo "Install Jupyter java kernel..."
curl -L https://github.com/SpencerPark/IJava/releases/download/v1.3.0/ijava-1.3.0.zip \
 -o ijava-kernel.zip &> /dev/null

unzip -q ijava-kernel.zip -d ijava-kernel \
 && cd ijava-kernel \
 && python3 install.py --sys-prefix &> /dev/null

echo "Install proxy for the java kernel"
# NOTE: required after changes to Google Colab defaults in Dec. 2022
# See https://stackoverflow.com/questions/74674688/google-colab-notebook-using-ijava-stuck-at-connecting-after-installation-ref/74821762#74821762

wget -qO- https://gist.github.com/SpencerPark/e2732061ad19c1afa4a33a58cb8f18a9/archive/b6cff2bf09b6832344e576ea1e4731f0fb3df10c.tar.gz | tar xvz --strip-components=1
python install_ipc_proxy_kernel.py --kernel=java --implementation=ipc_proxy_kernel.py


Update environment...
Install Java...
Install Jupyter java kernel...
Install proxy for the java kernel
e2732061ad19c1afa4a33a58cb8f18a9-b6cff2bf09b6832344e576ea1e4731f0fb3df10c/install_ipc_proxy_kernel.py
e2732061ad19c1afa4a33a58cb8f18a9-b6cff2bf09b6832344e576ea1e4731f0fb3df10c/ipc_proxy_kernel.py
Moving java kernel from /usr/share/jupyter/kernels/java...
Wrote modified kernel.json for java_tcp in /usr/share/jupyter/kernels/java_tcp/kernel.json
Installing the proxy kernel in place of java in /usr/share/jupyter/kernels/java
Installed proxy kernelspec: {"argv": ["/usr/bin/python3", "/usr/share/jupyter/kernels/java/ipc_proxy_kernel.py", "{connection_file}", "--kernel=java_tcp"], "env": {}, "display_name": "Java", "language": "java", "interrupt_mode": "message", "metadata": {}}
Proxy kernel installed. Go to 'Runtime > Change runtime type' and select 'java'


In [3]:
%maven ai.djl:api:0.22.1
%maven ai.djl:model-zoo:0.22.1
%maven ai.djl.mxnet:mxnet-model-zoo:0.22.1
%maven ai.djl.pytorch:pytorch-model-zoo:0.22.1
%maven ai.djl.onnxruntime:onnxruntime-engine:0.22.1
%maven ai.djl.pytorch:pytorch-engine:0.22.1
%maven ai.djl.opencv:opencv:0.22.1
%maven org.openpnp:opencv:4.7.0-0
%maven org.projectlombok:lombok:1.18.26
%maven org.slf4j:slf4j-simple:2.0.7
%maven org.json:json:20220320

In [5]:
import ai.djl.Application;
import ai.djl.MalformedModelException;
import ai.djl.inference.Predictor;
import ai.djl.modality.Classifications;
import ai.djl.modality.cv.Image;
import ai.djl.modality.cv.ImageFactory;
import ai.djl.modality.cv.output.DetectedObjects;
import ai.djl.modality.cv.transform.Resize;
import ai.djl.modality.cv.transform.ToTensor;
import ai.djl.modality.cv.translator.ImageClassificationTranslator;
import ai.djl.modality.cv.translator.ObjectDetectionTranslator;
import ai.djl.modality.cv.translator.YoloV5Translator;
import ai.djl.ndarray.NDList;
import ai.djl.repository.zoo.Criteria;
import ai.djl.repository.zoo.ModelNotFoundException;
import ai.djl.repository.zoo.ZooModel;
import ai.djl.repository.zoo.ModelZoo;
import ai.djl.training.util.ProgressBar;
import ai.djl.training.util.DownloadUtils;
import ai.djl.translate.Pipeline;
import ai.djl.translate.TranslateException;
import ai.djl.translate.Translator;
import ai.djl.translate.TranslatorContext;
import java.io.IOException;
import java.nio.file.Paths;
import java.util.List;
//import org.bytedeco.opencv.global.opencv_core;
//import nu.pattern.OpenCV;

In [ ]:
//%load ../utils/djl-imports

//import ai.djl.basicdataset.cv.classification.*;
//import ai.djl.metric.*;

In [7]:
DownloadUtils.download("https://drive.google.com/file/d/1GYbHystCnm1OQdfdLro7pW9NrO-PD-wX/", "build/pytorch_models/yoloV5/best_01jul2022_3.onnx", new ProgressBar());
DownloadUtils.download("https://drive.google.com/file/d/1fe1KZGG3l65Q5wZ39lcoS3eLB375Spp_/", "build/images/auto.png", new ProgressBar());
DownloadUtils.download("https://drive.google.com/file/d/1hEiotU_P-5tye26RI2xeEKt-5P_6gGN2/", "build/sinset/sinset.txt", new ProgressBar());

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [9]:
//OpenCV.loadLocally();
String imgPath = "/content/build/images/auto.png";
Pipeline pipeline = new Pipeline();
pipeline.add(new Resize(640, 640));
pipeline.add(new ToTensor());

ai.djl.translate.Pipeline@5f5bbc2e

In [11]:
Translator<Image, DetectedObjects> translator = YoloV5Translator
                .builder()
                .setPipeline(pipeline)
                .optSynsetUrl("file:///content/build/sinset/synset.txt")
                .optThreshold(0.1f)
                .build();

Criteria<Image, DetectedObjects> criteria
                = Criteria.builder()
                        .optApplication(Application.CV.OBJECT_DETECTION)
                        .setTypes(Image.class, DetectedObjects.class)
                        .optModelUrls("file:///content/build/pytorch_models/yoloV5/best_01jul2022_3.onnx")
                        .optTranslator(translator)
                        .optProgress(new ProgressBar())
                        .optEngine("OnnxRuntime")
                        .build();


In [13]:
for (ModelZoo modeloZoo : ModelZoo.listModelZoo()) {
  System.out.println(modeloZoo);
}

ai.djl.pytorch.zoo.PtModelZoo@4d298c9d
ai.djl.onnxruntime.zoo.OrtModelZoo@7977f6d7
ai.djl.basicmodelzoo.BasicModelZoo@4226de1f
ai.djl.mxnet.zoo.MxModelZoo@76929e41
ai.djl.repository.zoo.DefaultModelZoo@77deb96a


In [15]:
Image img = ImageFactory.getInstance().fromFile(Paths.get(imgPath));
try (ZooModel<Image, DetectedObjects> model = ModelZoo.loadModel(criteria)){
        Predictor<Image, DetectedObjects> predictor = model.newPredictor();
        //DetectedObjects result = predictor.predict(img);
        DetectedObjects Detected_dents = predictor.predict(img);
    }

EvalException: ignored

In [17]:
ZooModel<Image, DetectedObjects> model = criteria.loadModel();

Image img = ImageFactory.getInstance().fromFile(Paths.get(imgPath));
Predictor<Image, DetectedObjects> predictor = model.newPredictor();
DetectedObjects Detected_dents = predictor.predict(img);

//topK me quedo con la mejor predicción
List<Classifications.Classification> objs = Detected_dents.topK(1);
System.out.println(Detected_dents);

Loading:     100% |████████████████████████████████████████|


EvalException: ignored

In [ ]:
!find .. -name 'libopencv*'

../usr/lib/x86_64-linux-gnu/libopencv_flann.so.4.2
../usr/lib/x86_64-linux-gnu/libopencv_reg.a
../usr/lib/x86_64-linux-gnu/libopencv_freetype.so.4.2.0
../usr/lib/x86_64-linux-gnu/libopencv_video.so
../usr/lib/x86_64-linux-gnu/libopencv_quality.a
../usr/lib/x86_64-linux-gnu/libopencv_core.so
../usr/lib/x86_64-linux-gnu/libopencv_reg.so.4.2
../usr/lib/x86_64-linux-gnu/libopencv_bgsegm.so.4.2.0
../usr/lib/x86_64-linux-gnu/libopencv_superres.so
../usr/lib/x86_64-linux-gnu/libopencv_plot.so.4.2.0
../usr/lib/x86_64-linux-gnu/libopencv_fuzzy.so.4.2.0
../usr/lib/x86_64-linux-gnu/libopencv_dnn_objdetect.a
../usr/lib/x86_64-linux-gnu/libopencv_dpm.so.4.2
../usr/lib/x86_64-linux-gnu/libopencv_saliency.so
../usr/lib/x86_64-linux-gnu/libopencv_text.a
../usr/lib/x86_64-linux-gnu/libopencv_line_descriptor.so.4.2.0
../usr/lib/x86_64-linux-gnu/libopencv_video.so.4.2.0
../usr/lib/x86_64-linux-gnu/libopencv_fuzzy.a
../usr/lib/x86_64-linux-gnu/libopencv_xobjdetect.so.4.2.0
../usr/lib/x86_64-linux-gnu/libo